# Anomaly Detection - Project

In [1]:
# library imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

import env

import warnings
warnings.filterwarnings("ignore")

In [194]:
# access Codeup SQL Server for Curriculum Logs

url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
query = '''
SELECT date,
       path as endpoint,
       user_id,
       ip as source_ip,
       name as cohort,
       program_id,
       start_date,
       end_date
FROM logs
JOIN cohorts ON cohorts.id = logs.cohort_id;
'''
df = pd.read_sql(query, url)
df.head()

,date,endpoint,user_id,source_ip,cohort,program_id,start_date,end_date
0,2018-01-26,/,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
1,2018-01-26,java-ii,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2,2018-01-26,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
3,2018-01-26,slides/object_oriented_programming,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
4,2018-01-26,javascript-i/conditionals,2,97.105.19.61,Teddy,2,2018-01-08,2018-05-17


In [10]:
# change date to date_time format, and set the index to it

df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)

### Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [36]:
# Drop rows where cohort contains 'staff' (case-insensitive)
df = df[~df['cohort'].str.contains('staff', case=False)]

In [37]:
# Group the data by lesson and count users 
lesson_counts = df.groupby('endpoint')['user_id'].count()

# Sort the lessons by their counts
sorted_lessons = lesson_counts.sort_values(ascending=False)

# Print the top 10 lessons by count
print(sorted_lessons.head(10))

endpoint
/                           39514
toc                         16680
javascript-i                16386
search/search_index.json    16185
html-css                    11843
java-iii                    11773
java-ii                     10917
spring                      10480
jquery                      10124
mysql                        9716
Name: user_id, dtype: int64


In [38]:
# Group the data by both lesson and cohort, and count the number of times each lesson was referred to by each cohort
cohort_lesson_counts = df.groupby(['cohort', 'endpoint'])['user_id'].count()

In [39]:
# Loop through each cohort and compare its count for each lesson to the average count across all cohorts

for cohort in df['cohort'].unique():
    cohort_lessons = cohort_lesson_counts[cohort_lesson_counts.index.get_level_values('cohort') == cohort]

In [40]:
cohort_lessons

cohort  endpoint                                                                 
Oberon  /                                                                            131
        appendix                                                                      10
        appendix/code-standards/javascript                                             1
        appendix/coding-challenges                                                     1
        appendix/coding-challenges/amateur                                             2
        appendix/extra-challenges                                                      1
        appendix/extra-challenges/basic-challenges                                     3
        appendix/further-reading/javascript/array-splice                               1
        appendix/further-reading/javascript/hoisting                                   4
        appendix/git                                                                   1
        appendix/git/branchi

In [41]:
# Compute the count of each lesson across all cohorts
lesson_counts = df.groupby('endpoint')['user_id'].count()

# Filter out lessons with less than 100 total counts across all cohorts
valid_lessons = lesson_counts[lesson_counts >= 100].index

# Compute the count of each lesson for each cohort
cohort_lesson_counts = pd.pivot_table(df, values='user_id', index='cohort', columns='endpoint', aggfunc='count', fill_value=0)


In [42]:
cohort_lesson_counts

endpoint,%20https://github.com/RaulCPena,",%20https://github.com/RaulCPena",.git,.gitignore,.well-known/assetlinks.json,/,00_index,01_intro,02_listing_files,03_file_paths,...,web-design/ui/visuals,web-design/ux,web-design/ux/layout,web-design/ux/layout/.json,web-design/ux/purpose,web-dev-day-two,where,working-with-time-series-data,wp-admin,wp-login
cohort,,,,,,,,,,,,,,,,,,,,,
Andromeda,0,0,0,0,0,1174,0,0,0,0,...,11,0,6,0,8,0,0,0,0,0
Apex,0,0,0,0,0,1346,0,0,0,0,...,20,0,18,0,23,0,0,0,0,0
Apollo,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Arches,0,0,0,0,0,626,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Badlands,0,0,0,0,0,17,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bash,0,0,0,0,0,772,0,0,0,0,...,5,0,3,0,3,2,0,0,0,0
Bayes,0,0,0,0,0,1967,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Betelgeuse,0,0,0,0,0,955,0,0,0,0,...,30,0,38,0,32,0,0,0,0,0
Ceres,0,0,0,0,0,1653,0,0,0,0,...,34,0,34,1,28,0,0,0,0,0


In [43]:
# Filter out lessons with less than 100 counts
cohort_lesson_counts = cohort_lesson_counts.loc[:, (cohort_lesson_counts >= 100).any()]

In [44]:
cohort_lesson_counts

endpoint,/,1-fundamentals/1.1-intro-to-data-science,1-fundamentals/1.2-data-science-pipeline,1-fundamentals/1.3-pipeline-demo,1-fundamentals/2.2-excel-functions,1-fundamentals/2.3-visualization-with-excel,1-fundamentals/2.4-more-excel-features,1-fundamentals/AI-ML-DL-timeline.jpg,1-fundamentals/DataToAction_v2.jpg,1-fundamentals/modern-data-scientist.jpg,...,stats/probability-distributions,stats/simulation,storytelling/create,storytelling/understand,timeseries/acquire,timeseries/explore,timeseries/modeling-lesson1,timeseries/prep,timeseries/working-with-time-series-data,toc
cohort,,,,,,,,,,,,,,,,,,,,,
Andromeda,1174,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,638
Apex,1346,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,810
Apollo,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Arches,626,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,66
Badlands,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
Bash,772,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,404
Bayes,1967,640,221,130,153,175,115,648,230,650,...,12,13,2,3,10,8,25,13,7,9
Betelgeuse,955,9,1,0,1,0,0,8,1,9,...,0,0,0,0,0,0,0,0,0,610
Ceres,1653,7,0,0,0,0,0,7,0,7,...,0,0,0,0,0,0,0,0,0,911


In [45]:
# Compute the z-score for each lesson count across all cohorts
cohort_lesson_counts_z = (cohort_lesson_counts - cohort_lesson_counts.mean()) / cohort_lesson_counts.std()

In [148]:
# Set the threshold for the z-score
z_threshold = 6

In [149]:
# Loop through each lesson and check if at least one coho5rt has a z-score greater than the threshold
for lesson in cohort_lesson_counts_z.columns:
    if (cohort_lesson_counts_z[lesson] > z_threshold).any():
        print(f"At least one cohort referred to lesson {lesson} significantly more than other cohorts")

At least one cohort referred to lesson 1-fundamentals/2.2-excel-functions significantly more than other cohorts
At least one cohort referred to lesson 2-storytelling/project significantly more than other cohorts
At least one cohort referred to lesson 3-sql/1-mysql-overview significantly more than other cohorts
At least one cohort referred to lesson 3-sql/11-more-exercises significantly more than other cohorts
At least one cohort referred to lesson 3-sql/12-more-exercises significantly more than other cohorts
At least one cohort referred to lesson 4-python/7-working-with-files significantly more than other cohorts
At least one cohort referred to lesson 4-python/7.2-intro-to-matplotlib significantly more than other cohorts
At least one cohort referred to lesson 4-python/7.3-intro-to-numpy significantly more than other cohorts
At least one cohort referred to lesson 4-python/7.4.2-series significantly more than other cohorts
At least one cohort referred to lesson 4-python/7.4.3-dataframes 

In [136]:
# Find the lesson with the highest z-score
lesson_with_highest_zscore = cohort_lesson_counts_z.max().idxmax()

print(f"The lesson with the highest z-score is {lesson_with_highest_zscore}")

The lesson with the highest z-score is 4-python/7.4.2-series


In [137]:
# Find the lesson with the highest z-score
lesson_with_highest_zscore = cohort_lesson_counts_z.max().idxmax()

# Find the cohort with the highest z-score for the lesson with the highest z-score
cohort_with_highest_zscore = cohort_lesson_counts_z.loc[:, lesson_with_highest_zscore].idxmax()

print(f"The cohort with the highest contribution to the z-score for {lesson_with_highest_zscore} is {cohort_with_highest_zscore}")

The cohort with the highest contribution to the z-score for 4-python/7.4.2-series is Bayes


### Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 763299 entries, 2018-01-26 to 2021-04-21
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        763299 non-null  datetime64[ns]
 1   endpoint    763298 non-null  object        
 2   user_id     763299 non-null  int64         
 3   source_ip   763299 non-null  object        
 4   cohort      763299 non-null  object        
 5   program_id  763299 non-null  int64         
 6   start_date  763299 non-null  object        
 7   end_date    763299 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 52.4+ MB


In [53]:
df.head()

,date,endpoint,user_id,source_ip,cohort,program_id,start_date,end_date
date,,,,,,,,
2018-01-26,2018-01-26,/,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2018-01-26,2018-01-26,java-ii,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2018-01-26,2018-01-26,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2018-01-26,2018-01-26,slides/object_oriented_programming,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2018-01-26,2018-01-26,javascript-i/conditionals,2,97.105.19.61,Teddy,2,2018-01-08,2018-05-17


In [171]:
# Calculate total number of times each student accessed any endpoint during their active period
student_activity_counts = df[df.index < df['end_date']].groupby('user_id')['endpoint'].count()

# Filter out students who accessed the curriculum fewer than 10 times during their active period
inactive_students = student_activity_counts[student_activity_counts < 5].index.tolist()

# Print information about inactive students
inactive_student_data = df[df['user_id'].isin(inactive_students)][['user_id', 'cohort', 'start_date', 'end_date']].drop_duplicates()
print(inactive_student_data)

            user_id     cohort  start_date    end_date
date                                                  
2018-09-27      278  Voyageurs  2018-05-29  2018-10-11
2019-11-04      539     Europa  2019-11-04  2020-04-17
2020-12-07      832    Jupiter  2020-09-21  2021-03-30
2021-04-15      956     Oberon  2021-04-12  2021-10-01


In [73]:
# Calculate total number of times each student accessed each endpoint during their active period
inactive_student_endpoint_counts = df[df.index < df['end_date']].groupby(['user_id', 'endpoint']).size()

# Filter out inactive students who accessed the curriculum fewer than 5 times
inactive_students = student_activity_counts[student_activity_counts < 5].index.tolist()

# Calculate the number of times each inactive student accessed each endpoint during their active period
inactive_student_endpoint_counts = inactive_student_endpoint_counts.loc[inactive_students]

# Remove the '/' endpoint from the results
inactive_student_endpoint_counts = inactive_student_endpoint_counts.reset_index().query('endpoint != "/"').set_index(['user_id', 'endpoint'])

# Print the results
print(inactive_student_endpoint_counts)

                             0
user_id endpoint              
278     java-ii/arrays       2
        java-ii/collections  1
832     html-css             1
        javascript-i         1


In [170]:
# Group the DataFrame by student and count the number of times each student accessed the curriculum
student_access_counts = df.groupby('user_id')['endpoint'].count()

# Calculate the mean and standard deviation of the access counts
mean_access_count = student_access_counts.mean()
std_access_count = student_access_counts.std()

# Calculate the Z-scores for each student's access count
z_scores = (student_access_counts - mean_access_count) / std_access_count

# Identify any Z-scores that are less than -1.05 standard deviations from the mean
outliers = z_scores[z_scores < -1.05]

# Print the outliers and any available information about these students
if not outliers.empty:
    print(f"Found {len(outliers)} outlier students:")
    for student_id, z_score in outliers.iteritems():
        print(f"Student ID {student_id}: Z-score = {z_score:.2f}")
        # Here you could retrieve additional information about the student from your data
else:
    print("No outlier students found.")

Found 4 outlier students:
Student ID 165: Z-score = -1.05
Student ID 212: Z-score = -1.05
Student ID 574: Z-score = -1.05
Student ID 593: Z-score = -1.05


In [196]:
df[df['user_id'] == 165]

,date,endpoint,user_id,source_ip,cohort,program_id,start_date,end_date
35018,2018-03-30,index.html,165,24.243.3.200,Niagara,2,2016-10-26,2017-03-09


In [197]:
df[df['user_id'] == 272]

,date,endpoint,user_id,source_ip,cohort,program_id,start_date,end_date
121776,2018-09-17,/,272,97.105.19.58,Xanadu,2,2018-09-17,2019-02-08
121826,2018-09-17,toc,272,97.105.19.58,Xanadu,2,2018-09-17,2019-02-08
122113,2018-09-18,toc,272,97.105.19.58,Xanadu,2,2018-09-17,2019-02-08
122125,2018-09-18,html-css/introduction,272,97.105.19.58,Xanadu,2,2018-09-17,2019-02-08
122129,2018-09-18,html-css/elements,272,97.105.19.58,Xanadu,2,2018-09-17,2019-02-08
...,...,...,...,...,...,...,...,...
510688,2020-05-10,/,272,72.181.119.126,Xanadu,2,2018-09-17,2019-02-08
510689,2020-05-10,java-iii,272,72.181.119.126,Xanadu,2,2018-09-17,2019-02-08
510690,2020-05-10,java-ii,272,72.181.119.126,Xanadu,2,2018-09-17,2019-02-08
510691,2020-05-10,javascript-ii,272,72.181.119.126,Xanadu,2,2018-09-17,2019-02-08


In [198]:
df[df['user_id'] == 574]

,date,endpoint,user_id,source_ip,cohort,program_id,start_date,end_date
415874,2020-01-30,/,574,24.160.4.6,Voyageurs,2,2018-05-29,2018-10-11
415875,2020-01-30,toc,574,24.160.4.6,Voyageurs,2,2018-05-29,2018-10-11


In [199]:
df[df['user_id'] == 593]

,date,endpoint,user_id,source_ip,cohort,program_id,start_date,end_date
431000,2020-02-17,index.html,593,167.24.104.150,Lassen,1,2016-07-18,2016-11-10


### What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 763299 entries, 2018-01-26 to 2021-04-21
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        763299 non-null  datetime64[ns]
 1   endpoint    763298 non-null  object        
 2   user_id     763299 non-null  int64         
 3   source_ip   763299 non-null  object        
 4   cohort      763299 non-null  object        
 5   program_id  763299 non-null  int64         
 6   start_date  763299 non-null  object        
 7   end_date    763299 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 52.4+ MB


In [75]:
df.head()

,date,endpoint,user_id,source_ip,cohort,program_id,start_date,end_date
date,,,,,,,,
2018-01-26,2018-01-26,/,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2018-01-26,2018-01-26,java-ii,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2018-01-26,2018-01-26,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2018-01-26,2018-01-26,slides/object_oriented_programming,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2018-01-26,2018-01-26,javascript-i/conditionals,2,97.105.19.61,Teddy,2,2018-01-08,2018-05-17


In [85]:
# Create a new DataFrame containing only the endpoints accessed by graduates after they have graduated
graduated_endpoints = df[(df['end_date'] <= df.index) & (df['program_id'].notna())]

# Group the DataFrame by program and endpoint, and count the number of times each endpoint is accessed
graduated_endpoint_counts = graduated_endpoints.groupby(['program_id', 'endpoint']).size()

# Print the results, sorted by program and then by count
print(graduated_endpoint_counts.groupby('program_id').apply(lambda x: x.sort_values(ascending=False)))

program_id  program_id  endpoint                                   
1           1                                                          1681
                        index.html                                     1011
                        javascript-i                                    736
                        html-css                                        542
                        spring                                          501
                                                                       ... 
3           3           viz.py                                            1
4           4           content/html-css                                  2
                                                                          1
                        content/html-css/gitbook/images/favicon.ico       1
                        content/html-css/introduction.html                1
Length: 1971, dtype: int64


In [86]:
top_counts = graduated_endpoint_counts.groupby(['program_id', 'endpoint']).size().reset_index(name='count').sort_values(by=['program_id', 'count'], ascending=[True, False])
print(top_counts.groupby('program_id').head(1))


      program_id endpoint  count
0              1               1
710            2               1
1530           3               1
1967           4               1


In [87]:
# Remove leading and trailing slashes from endpoint strings
df['endpoint'] = df['endpoint'].str.strip('/')

# Filter out endpoints that are empty strings or contain only whitespace
df = df[df['endpoint'].str.strip() != '']

# Filter the DataFrame to only include graduates
graduated_endpoints = df[(df['end_date'] <= df.index) & (df['program_id'].notna())]

# Group the DataFrame by program and endpoint, and count the number of times each endpoint is accessed
graduated_endpoint_counts = graduated_endpoints.groupby(['program_id', 'endpoint']).size()

# Print the results, sorted by program and then by count
print(graduated_endpoint_counts.groupby('program_id').apply(lambda x: x.sort_values(ascending=False).head()))


program_id  program_id  endpoint                                      
1           1           index.html                                        1011
                        javascript-i                                       736
                        html-css                                           542
                        spring                                             501
                        java-iii                                           479
2           2           javascript-i                                      2416
                        spring                                            2368
                        search/search_index.json                          2222
                        html-css                                          1861
                        java-i                                            1831
3           3           search/search_index.json                           493
                        sql/mysql-overview                  

### Which lessons are least accessed?

In [93]:
df.head()

,date,endpoint,user_id,source_ip,cohort,program_id,start_date,end_date
date,,,,,,,,
2018-01-26,2018-01-26,java-ii,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2018-01-26,2018-01-26,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2018-01-26,2018-01-26,slides/object_oriented_programming,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2018-01-26,2018-01-26,javascript-i/conditionals,2,97.105.19.61,Teddy,2,2018-01-08,2018-05-17
2018-01-26,2018-01-26,javascript-i/loops,2,97.105.19.61,Teddy,2,2018-01-08,2018-05-17


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 723785 entries, 2018-01-26 to 2021-04-21
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        723785 non-null  datetime64[ns]
 1   endpoint    723784 non-null  object        
 2   user_id     723785 non-null  int64         
 3   source_ip   723785 non-null  object        
 4   cohort      723785 non-null  object        
 5   program_id  723785 non-null  int64         
 6   start_date  723785 non-null  object        
 7   end_date    723785 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 49.7+ MB


In [96]:
# Group the DataFrame by lesson and count the number of times each lesson is accessed
lesson_counts = df.groupby('endpoint').size()

# Sort the results by count in ascending order
lesson_counts_sorted = lesson_counts.sort_values()

# Print the 10 least accessed lessons
print(lesson_counts_sorted.head(10))

endpoint
%20https://github.com/RaulCPena                                1
cohorts/24/capstone-all                                        1
cohorts/24/grades                                              1
console-io                                                     1
content/appendix/control-structures.html                       1
content/appendix/javascript/functions/controllers.html         1
content/appendix/javascript/functions/models.html              1
content/appendix/javascript/functions/scope.html               1
content/appendix/javascript/functions/templating.html          1
content/appendix/javascript/javascript/functions/scope.html    1
dtype: int64


In [113]:
results = lesson_counts_sorted[lesson_counts_sorted == 1]

In [119]:
results.sum()

419

In [118]:
for index in results.index:
    print(index)

%20https://github.com/RaulCPena
cohorts/24/capstone-all
cohorts/24/grades
console-io
content/appendix/control-structures.html
content/appendix/javascript/functions/controllers.html
content/appendix/javascript/functions/models.html
content/appendix/javascript/functions/scope.html
content/appendix/javascript/functions/templating.html
content/appendix/javascript/javascript/functions/scope.html
content/conditionals.html
content/control-structures-ii
content/examples/appendix
content/examples/constructors-destructors.html
content/examples/examples/examples/gitbook/images/favicon.ico
content/examples/examples/gitbook/images/favicon.ico
content/examples/git/gitbook/images/favicon.ico
content/examples/html
coding-challenges
content/examples/html/forms.html
codeupexercises
code-standards/javascript
capsones/151
capstone/50
capstone/52
capstone/53
capstone/54
capstone/55
capstone/jupiter
capstone/team
capstone/teams
capstones
case-statements
classification/explore-old
classification/knn.md
cli-0

In [110]:
lesson_access_counts = df.groupby('endpoint')['user_id'].nunique()
lessons_barely_accessed = lesson_access_counts[lesson_access_counts == 1]
lessons_barely_accessed

endpoint
%20https://github.com/RaulCPena     1
,%20https://github.com/RaulCPena    1
.git                                1
.well-known/assetlinks.json         1
00_index                            1
                                   ..
web-design/ux/layout/.json          1
web-dev-day-two                     1
where                               1
wp-admin                            1
wp-login                            1
Name: user_id, Length: 553, dtype: int64

### Which lesson appears to attract the most traffic consistently across cohorts (per program)? 

In [123]:
# Group the DataFrame by program and lesson, and sum the count of each lesson across all cohorts
program_lesson_counts = df.groupby(['program_id', 'endpoint'])['user_id'].sum()

# Sort the results by program and count
program_lesson_counts_sorted = program_lesson_counts.groupby('program_id').apply(lambda x: x.sort_values(ascending=False))

# Print the program and lesson with the highest count for each program
for program in program_lesson_counts_sorted.index.levels[0]:
    print(f"\nProgram: {program}")
    print(program_lesson_counts_sorted[program].head(10))


Program: 1
program_id  endpoint        
1           javascript-i        79944
            index.html          62165
            html-css            57503
            java-i              51383
            content/html-css    43753
            javascript-ii       41641
            java-ii             39588
            jquery              39546
            spring              33933
            java-iii            33124
Name: user_id, dtype: int64

Program: 2
program_id  endpoint                
2           toc                         9088058
            javascript-i                7837855
            search/search_index.json    7389507
            html-css                    5609408
            java-iii                    5167688
            java-ii                     4819362
            jquery                      4582666
            spring                      4271962
            mysql                       4190224
            java-i                      4078856
Name: user_id, dtype: 